# lab-10_3_2_MNIST-CNN with Visdom
##### JaehyunJeong

In [24]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

import visdom

In [25]:
vis = visdom.Visdom()
vis.close(env="main")

Setting up a new session...


''

## define loss_tracker

In [26]:
def loss_tracker(loss_plot, loss_value, num):
    vis.line(X=num,
            Y=loss_value,
            win=loss_plot,
            update='append')

## define model

In [27]:
# 디바이스 및 시드값 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)
    
    
# 파라미터 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 32

In [28]:
# 데이터셋 준비
mnist_train = dsets.MNIST(root='./MNIST_data/',
                         train = True,
                         transform=transforms.ToTensor(),
                         download=True)

mnist_test = dsets.MNIST(root='./MNIST_data/',
                        train=False,
                        transform = transforms.ToTensor(),
                        download=True)

data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size = batch_size,
                                         shuffle = True,
                                         drop_last = True)

In [29]:
# 모델 생성
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5

        # conv layer
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1), # 입력 채널 1, 출력 채널(필터 갯수) 32
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # conv layer
        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64,kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # conv layer
        self.layer3 = nn.Sequential(
            nn.Conv2d(64,128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        )

        self.fc1 = nn.Linear(4*4*128, 625, bias=True) # 4*4 사이즈의 맵이 64개(입력), 625(출력)
        torch.nn.init.xavier_uniform_(self.fc1.weight) # fc층은 가중치 초기화 적용

        # fc layer
        self.layer4 = nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p = self.keep_prob)
        )

        self.fc2 = torch.nn.Linear(625, 10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x) # conv
        out = self.layer2(out) # conv
        out = self.layer3(out) # conv
        out = out.view(out.size(0), -1) # flatten
        out = self.layer4(out) # fc
        out = self.fc2(out) # fc
        return out

In [30]:
model = CNN().to(device)
# 파라미터 정의
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [31]:
# 사분면 그리기
loss_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))

## training with loss_tracker

In [32]:
# training
total_batch = len(data_loader) # 600개의 배치 묶음
model.train() # dropout = True

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader : # (image, label)
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)

        cost = criterion(hypothesis, Y) # 한 배치묶음(100개의 이미지)의 평균 loss(cost)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch : {:>2}] cost = {:.9}'.format(epoch+1, avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch])) # loss tracker
print('Learning Finished!')

[Epoch :  1] cost = 0.155277029
[Epoch :  2] cost = 0.0511719361
[Epoch :  3] cost = 0.0370592289
[Epoch :  4] cost = 0.0295228884
[Epoch :  5] cost = 0.0231849477
[Epoch :  6] cost = 0.0188196134
[Epoch :  7] cost = 0.0175009556
[Epoch :  8] cost = 0.0149418386
[Epoch :  9] cost = 0.0150751816
[Epoch : 10] cost = 0.0110050235
[Epoch : 11] cost = 0.0124356896
[Epoch : 12] cost = 0.00971969031
[Epoch : 13] cost = 0.0113638993
[Epoch : 14] cost = 0.0101935379
[Epoch : 15] cost = 0.00978853367
Learning Finished!


In [37]:
# test model and check accuracy
with torch.no_grad():
    model.eval() # dropout = False

    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28,28).float().to(device) # 배치사이즈*
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('accuracy : {:.5f}'.format(accuracy.item()))

accuracy : 0.95020
